In [ ]:
import requests
from google.colab import files
import time
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url) #makes a request to server, downloads html page for this
# # data.text #gives the long html - not using this, use page instead
# #going to grab <a tag from html for every squad in the league to get their stats page
# #will parse html link with beautifulsoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text) #pass in the html
# #looking at the page, all the data we want is within the table
# #from the table, we can take all the <a tags
time.sleep(2)
standings_table = soup.select('table.stats_table')[0] #css selector, select the table, class name is stats_table, [0] makes sure the first one is selected
links = standings_table.find_all('a') #use find_all to get all of the tags
links = [l.get("href") for l in links] #makes a list, finding the href property of each link
# #in form <a href="...", and we need the href of all of the <a tags
links = [l for l in links if '/squads/' in l] #filters so only squad links in the list
# #however links dont have the full link with domain, so need to use format string to fix that
team_urls = [f"https://fbref.com{l}" for l in links]
team = team_urls[0] #for now working with just 1 team
data = requests.get(team) #get the html of that url
import pandas as pd
match_table = pd.read_html(data.text, match="Scores & Fixtures") #scanning all tables on page and choosing one with the "match" string in it
#match_table[0] -- getting the first occurance
soup2 = BeautifulSoup(data.text) #pass in data html
links = soup2.find_all('a') #have to find all then from there get the one we want
links = [l.get("href") for l in links] #use list comprehension to get hrefs from a tags
links = [l for l in links if l and 'all_comps/shooting/' in l] #chooses the shooting url from all of the data
data = requests.get(f"https://fbref.com{links[0]}") #will give us the data
shooting = pd.read_html(data.text, match="Shooting")[0] #scanning the page and choosing the first one [0] with "Shooting" in it
#shooting.head() #see first 5 rows with head method
shooting.columns = shooting.columns.droplevel() #gets rid of the multilevel index, not needed for pandas
team_data = match_table[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")

time.sleep(1)
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
time.sleep(1)
years = [i for i in range(2024,2017,-1)]
all_matches = []
for year in years:
  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  previous_season = soup.select("a.prev")[0].get("href")
  standings_url = f"https://fbref.com{previous_season}"
  for team in team_urls:
    name = team.split("/")[-1].replace("-Stats","").replace("-"," ") #turns link into a team name, like Manchester City, [-1] gets last
    data = requests.get(team)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]

    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try: #skipping over teams where shooting stats arent avaiable
      team_data = matches.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")
    except ValueError:
      continue

    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = year
    team_data["Team"] = name
    all_matches.append(team_data)
    time.sleep(5) #slow down scraping so we dont get blocked

match_df = pd.concat(all_matches) #turns list of data into 1 dataframe
match_df.to_csv('data.csv')
files.download('data.csv')